## Bereinigung und Erweiterung der Daten

Hier werden die Daten der Expertenprofile für die Experimente vorbereitet.

Zuerst werden die Daten, da wo es notwendig ist, bereinigt und normalisiert.

Anschließend werden die Daten durch weitere Datenfelder ergänzt und erweitert:

1. Hinzufügen einer Volltextbeschreibung vom ganzen Profil.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH = "drive/MyDrive/Uni/Master/Masterthesis/Data/"

In [ ]:
import pandas as pd

In [ ]:
# Load anonymized profile data from JSON file.
expert_df = pd.read_json(PATH + "anonymized-profiles.json")

# Display first rows for initial inspection.
print(expert_df.head())

                                            branches companyLocationCity  \
0  [Technologie, Medien & Werbung, Unternehmensdi...          Lennestadt   
1                     [Unternehmensdienstleistungen]         Lüdenscheid   
2  [Automobilindustrie, Maschinenbau und Industri...         Lüdenscheid   
3  [Unternehmensdienstleistungen, Investor (Eigen...         Lüdenscheid   
4                      [Logistik, Konsum und Handel]         Ludenscheid   

   companyLocationStreet  companyLocationZip            companyName  \
0      Hagener Straße 64             57368.0        lenne.Tech GmbH   
1   Sauerfelder Straße 5             58511.0            RegioKonneX   
2   Sauerfelder Straße 5             58511.0            RegioKonneX   
3  Sauerfelder Straße 4              58511.0            RegioKonneX   
4   Wefelshohler Str 48a             58511.0  Karl Koerschulte GmbH   

       companyTypes      companyWebsite  \
0  [Dienstleistung]  https://lenne.tech   
1  [Dienstleistung]  www.regio

In [ ]:
len(expert_df)

347

In [ ]:
# expert_df in eine CSV schreiben
#expert_df.to_csv(PATH + "expert_profiles_raw.csv", index=False)

# Cleaning

In [ ]:
import re

def clean_value_or_list(value):
    """
    Reinigt einen Wert oder eine Liste von Werten, indem numerische Werte
    und ungültige Strings entfernt werden.

    Args:
        value: Der zu reinigende Wert (String oder Liste).

    Returns:
        Der gereinigte Wert oder die gereinigte Liste.
    """

    # Validate string quality based on heuristic rules.
    def is_invalid_string(s):
        # Reject non-strings or empty strings.
        if not isinstance(s, str) or len(s.strip()) == 0:
            return True

        # Reject pure numeric strings.
        if s.isdigit():
            return True

        # Filter very short or very long values.
        if len(s) < 2 or len(s) > 100:
            return True

        # Reject strings with low character diversity (e.g., "aaaaaa").
        if len(set(s.lower())) < len(s) * 0.2:
            return True

        # Reject strings with high ratio of non-alphanumeric characters.
        non_alnum_ratio = len(re.findall(r'[^a-zA-Z0-9]', s)) / len(s)
        if non_alnum_ratio > 0.5:
            return True

        # Reject mixed alphanumeric patterns often indicating random IDs.
        if re.search(r'\d+[a-zA-Z]+|[a-zA-Z]+\d+', s):
            return True

        return False


    # Handle list input.
    if isinstance(value, list):
        cleaned_list = []
        for item in value:
            # Exclude numeric values and invalid strings.
            if not (isinstance(item, (int, float)) or is_invalid_string(item)):
                cleaned_list.append(item)
        return cleaned_list

    # Handle scalar input.
    else:
        # Map invalid or numeric values to None.
        if isinstance(value, (int, float)) or is_invalid_string(value):
            return None
        else:
            return value


# Example usage.
print(clean_value_or_list("16099437417"))
print(clean_value_or_list("Abteilungsleiter Wertpapier-Kompetenzcenter"))
print(clean_value_or_list("360° Fotografin"))
print(clean_value_or_list("GF"))
print(clean_value_or_list("Beraterin/Coach/Trainerin"))
print(clean_value_or_list(["abc", "123", "def", "456xyz", 789]))
print(clean_value_or_list(["valid string", "another valid string"]))

None
Abteilungsleiter Wertpapier-Kompetenzcenter
360° Fotografin
GF
Beraterin/Coach/Trainerin
['abc', 'def']
['valid string', 'another valid string']


In [ ]:
# Columns subject to value normalization/cleaning.
columns_to_clean = ["jobTitle", "branches", "skills"]

# Apply cleaning function column-wise.
for col in columns_to_clean:
    expert_df[col] = expert_df[col].apply(clean_value_or_list)

# Optional: preview cleaned columns.
print(expert_df[columns_to_clean].head())

                                      jobTitle  \
0                           Softwareentwickler   
1                Administrator, Projektmanager   
2  Relationship-Manager (Gründung & Nachfolge)   
3        Firmenkundenbetreuung - Startup Coach   
4                              Geschäftsführer   

                                            branches  \
0  [Technologie, Medien & Werbung, Unternehmensdi...   
1                     [Unternehmensdienstleistungen]   
2  [Automobilindustrie, Maschinenbau und Industri...   
3  [Unternehmensdienstleistungen, Investor (Eigen...   
4                      [Logistik, Konsum und Handel]   

                                              skills  
0  [Softwareentwicklung, Web-Entwicklung, IT-Syst...  
1  [Financial Management, Cybersecurity, Finance,...  
2  [Benchmarking, Budget Planning, Budgeting, Bus...  
3  [Business Planning, Business Development, Coac...  
4  [Logistik, Supply Chain Management, Controllin...  


In [ ]:
# expert_df zurück in JSON schreiben
# expert_df.to_json(PATH + "expert_profiles_cleaned.json", orient='records', indent=4)

In [ ]:
# expert_df in eine CSV schreiben
# expert_df.to_csv(PATH + "expert_profiles_cleaned.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd

def is_empty(value):
    """
    Helferfunktion, um zu prüfen, ob ein Wert leer ist (String, Liste, None, NaN).
    """
    # Handle explicit None.
    if value is None:
        return True

    # Empty or whitespace-only strings.
    if isinstance(value, str):
        return not value.strip()

    # Empty lists.
    if isinstance(value, list):
        return not bool(value)

    # Explicit float NaN handling.
    if isinstance(value, np.float64) and np.isnan(value):
        return True

    # Generic pandas missing value check (NaN, NaT, etc.).
    if pd.isna(value):
        return True

    return False


# Condition 1: Profiles without skills.
no_skills_condition = expert_df['skills'].apply(is_empty)

# Condition 2: Profile has either a description or a projectsDescription.
has_description_condition = expert_df['description'].apply(lambda x: not is_empty(x))
has_projects_description_condition = expert_df['projectsDescription'].apply(lambda x: not is_empty(x))

# Combine conditions.
filtered_profiles = expert_df[
    no_skills_condition &
    (has_description_condition | has_projects_description_condition)
]

# Output summary and debug sample if applicable.
print(
    "Anzahl der Profile ohne Skills, aber mit Description oder ProjectsDescription: "
    f"{len(filtered_profiles)}"
)

if not filtered_profiles.empty:
    print("--- Debugging der ersten paar angeblich problematischen Profile ---")

    # Inspect first few matches for validation.
    for idx, row in filtered_profiles.head(5).iterrows():
        print(f"ID: {row['id']}")
        print(f"  Skills: {row['skills']}, ist leer: {is_empty(row['skills'])}")
        print(f"  Description: {repr(row['description'])}, ist nicht leer: {not is_empty(row['description'])}")
        print(f"  ProjectsDescription: {repr(row['projectsDescription'])}, ist nicht leer: {not is_empty(row['projectsDescription'])}")
        print(
            "  Gesamte Bedingung: "
            f"{is_empty(row['skills']) and (not is_empty(row['description']) or not is_empty(row['projectsDescription']))}"
        )
        print("------------------------------------------------------------------")

    display(filtered_profiles[['id', 'jobTitle', 'skills', 'description', 'projectsDescription']])
else:
    print("Es wurden keine solchen Profile gefunden.")

Anzahl der Profile ohne Skills, aber mit Description oder ProjectsDescription: 22
--- Debugging der ersten paar angeblich problematischen Profile ---
ID: 65b12e3f3897d6f0e651a423
  Skills: [], ist leer: True
  Description: None, ist nicht leer: False
  ProjectsDescription: 'Wir machen euer Firmenevent zum Erlebnis. Durch die Erfahrung aus unseren eigenen Events, wie den italienischen Nächsten und zB dem Booster Konzert in Lüdenscheid mit bis zu 3.000 Besuchern, und aus meinem Stanz-Biegeunternehmen Biegeform, weiß ich genau, worauf es bei Firmenevents ankommt und was man sich sowohl als Besucher, als auch als Chef, der alles bezahlen muss wünscht! Dazu bekommt ihr alles aus einer Hand von uns - also ein Ansprechpartner und alles läuft!', ist nicht leer: True
  Gesamte Bedingung (no_skills_condition & (has_description_condition | has_projects_description_condition)): True
------------------------------------------------------------------
ID: 65d7966a61580d8db501bf98
  Skills: [], ist le

,id,jobTitle,skills,description,projectsDescription
28,65b12e3f3897d6f0e651a423,Unternehmer,[],None,Wir machen euer Firmenevent zum Erlebnis. Durc...
92,65d7966a61580d8db501bf98,Head of Coffeeining,[],Ich bin Lulu (Sammy Kloss)\nKopf der Lulus Cof...,None
121,66091ec3b5bb2b1c9a92dca6,Sozial Media Agentur,[],Wir sind deine Marketing Agentur für den Gesun...,NaN
138,6628c74db5bb2b1c9a999254,CEO & Co-Founder,[],Ich bin gebürtiger Iserlohn und habe gemeinsam...,
157,664f04f0f48ebd8f3f4ac0b9,Allianz Mitarbeiter Bereich Anlage und Betrieb...,[],Unterstützung von Unternehmen zur Mitarbeiterf...,None
172,6661ab1cd89259daa708b3d3,Researcher,[],None,Green-AI Hub Mittelstand\n\nWir unterstützen a...
177,666803c5d89259daa70a14f6,Finanzcoach,[],Ich verkaufe Versicherungen…. und Vieles mehr!...,factoring\nleasing\nFinanzierung\nAbrechnung\n...
214,66a7aad83c04b3996296e0b6,Geschäftsführender Gesellschafter,[],"Hi, mein Name ist Sky Schottmann (24) und ich ...",Wir vernetzen die GEN Z mit Unternehmen und ge...
224,66bf2fb60e9f33c69a8b1d52,Geschäftsführer,[],None,Präzisionsdreh- und Rohrteile aus Metall
234,66d72488528e751260e5d6c3,Geschäftsführer und Entwickler von ExpChat,[],Entdecken Sie ExpChat – das modernste KI-Tool ...,NaN


In [ ]:
import pandas as pd

# Reuse the is_empty function defined previously in VOBm36L8jM6w
# Create boolean series for each condition
has_skills = expert_df['skills'].apply(lambda x: not is_empty(x))
has_description = expert_df['description'].apply(lambda x: not is_empty(x))
has_projects_description = expert_df['projectsDescription'].apply(lambda x: not is_empty(x))

# Combine conditions: at least one of them must be True
# This means: (skills are NOT empty) OR (description is NOT empty) OR (projectsDescription is NOT empty)
combined_condition = has_skills | has_description | has_projects_description

# Filter the DataFrame
filtered_for_enrichment_df = expert_df[combined_condition]

# Define the new path for the cleaned and filtered data
NEW_CLEANED_DATA_PATH = PATH + "expert_profiles_ready_for_enrichment.json"

# Save the filtered DataFrame to a JSON file
filtered_for_enrichment_df.to_json(NEW_CLEANED_DATA_PATH, orient='records', indent=4, force_ascii=False)

print(f"Originally {len(expert_df)} profiles. Filtered {len(filtered_for_enrichment_df)} experts based on the new criteria.")
print(f"Saved to {NEW_CLEANED_DATA_PATH}")

Originally 347 profiles. Filtered 307 experts based on the new criteria.
Saved to drive/MyDrive/Uni/Master/Masterthesis/Data/expert_profiles_ready_for_enrichment.json


# Enrichment

- Hier sollen dir Daten durch eine Volltextbeschreibung der Profile ergänzt werden

### Define llm components

In [ ]:
# install langchain
!pip install -U langchain
!pip install -U "langchain[openai]"

In [ ]:
# imports
import os
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# add api key for OPENAI
os.environ["OPENAI_API_KEY"] = input("Bitte gib deinen OpenAI API Key ein: ")
print("API Key erfolgreich gesetzt!")

In [ ]:
model = ChatOpenAI(
    model="gpt-5-nano-2025-08-07",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
SYSTEM_PROMPT = """
Deine Aufgabe ist es, aus einem JSON-Objekt, das einen Experten beschreibt, eine präzise und gut lesbare Volltextbeschreibung zu generieren.
Die Beschreibung soll die Fähigkeiten und Tätigkeiten des Experten kompakt zusammenfassen.

Datenquellen und Gewichtung:
Die relevanten Informationen stammen aus folgenden Datenfeldern:

Aktive Felder (direkt beschreibende Fähigkeiten, Hauptfokus):
- skills
- projectsDescription
- description

Passive Felder (indirekt beschreibende Fähigkeiten, Nebenfokus):
- branches
- jobTitle

Aktive Felder liefern konkrete Hinweise auf das, was der Experte tatsächlich kann oder tut.
Passive Felder liefern kontextuelle Informationen darüber, wo oder in welchem Rahmen die Fähigkeiten eingesetzt werden.
Die Inhalte aus den aktiven Feldern stehen im Vordergrund. Informationen aus den passiven Feldern sollen ergänzend und unterstützend erwähnt werden.

Anforderungen an die Ausgabe:
1. Verwende ausschließlich Informationen, die im JSON-Objekt vorhanden sind.
   Keine Halluzinationen oder frei erfundenen Inhalte. Fehlende oder leere Felder sollen ignoriert werden.
2. Erstelle zusätzlich eine Liste der verwendeten Datenfelder, diese Datenfelder dürfen nicht Null sein! Jedes Feld, das verwendet wurde, darf nur einmal in der Liste erscheinen!
   Beispiel: ["skills", "jobTitle", "projectsDescription", "branches", "description"]
3. Stil der Volltextbeschreibung: Knapp, präzise und flüssig formuliert. Im Fließtext, keine Aufzählungen. Fokus auf Fähigkeiten, Aufgaben und Anwendungsbereiche. Der Name des Experten darf im Text vorkommen, wenn vorhanden!

Beispiel 1:
```
  Eingabe:
  {{
    "branches": ["Reisen und Unterhaltung"],
    "jobTitle": "",
    "description": ""
    "projectsDescription": "- Organisation und Durchführung von Touren und Reisen, die nicht von der Stange kommen.\\n- Unikate.\\n- Organisation von eigenen und Kunden-Veranstaltungen.\\nBereiche: Kultur, Literatur, Konzerte; Sport- und Abenteuer, Motorrad- und Wander-Touren.",
    "skills": ["Projektmanagement", "Design", "Business Development", "Change Management", "Content Management", "Adobe Creative Cloud"]
  }}

  Ausgabe:
  Diese Expertin verfügt über ausgeprägte Fähigkeiten in Projektmanagement, Business Development und Change Management.
  Sie besitzt fundierte Kenntnisse im Design und Content Management, insbesondere im Umgang mit der Adobe Creative Cloud.
  In ihren Projekten organisiert und realisiert sie maßgeschneiderte Touren, Reisen und Veranstaltungen in den Bereichen Kultur, Literatur, Konzerte sowie Sport- und Abenteuertouren.
  Ihre Arbeitsschwerpunkte liegen im Bereich Reisen und Unterhaltung.

  Verwendete Felder:
  ["skills", "projectsDescription", "branches"]
```

Beispiel 2:
```
  Eingabe:
  {{
    "branches": [],
    "jobTitle": "Social Media Expert",
    "description": "Seit über 15 Jahren begleite ich Marketing-Projekte von der Konzeption bis zur Anwendung.",
    "projectsDescription": "Social Media Marketing, Inhaltsproduktion, Mitarbeitergewinnung, Webhosting, Webdesign, Webseiten Wartung",
    "skills": ["Marketing", "Recruiting", "Social Media Marketing", "Web-Entwicklung", "Benutzeroberflächen-Design", "Customer Relationship Management", "Google Analytics", "Marktforschung", "Stakeholder Management", "Content Management"]
  }}

  Ausgabe:
  Dieser Social Media Expert verfügt über umfassende Kompetenzen in Social Media Marketing, Recruiting und Content Management.
  Er begleitet seit über 15 Jahren Marketing-Projekte von der Konzeption bis zur Umsetzung und verfügt über fundierte Kenntnisse in Web-Entwicklung, Benutzeroberflächen-Design sowie Customer Relationship Management und Google Analytics.
  Darüber hinaus ist er erfahren in Marktforschung und Stakeholder Management und setzt diese Fähigkeiten gezielt zur Entwicklung wirksamer Kommunikationsstrategien ein.

  Verwendete Felder:
  ["skills", "projectsDescription", "jobTitle", "description"]
```

"""


In [ ]:
HUMAN_PROMPT = """
Erstelle basierend auf folgendem JSON-Objekt eine präzise Volltextbeschreibung des Experten.

Halte dich dabei strikt an die im Systemprompt beschriebenen Regeln:
- Fokus auf die aktiven Felder (skills, projectsDescription, description)
- Passive Felder (branches, jobTitle) nur ergänzend erwähnen
- Keine Halluzinationen, nur vorhandene Daten verwenden
- Gib zusätzlich eine Liste der verwendeten Datenfelder aus

Hier ist das JSON-Objekt:
{expert}
"""

In [ ]:
from typing import List, Literal

# Pydantic model representing the generated full-text profile output.
class Fulltext(BaseModel):
    """Ausgabe der Volltextgenerierung"""

    # Consolidated full-text representation of an expert profile.
    # Combines structured fields into a single searchable text.
    full_text: str = Field(
        description=(
            "Eine Volltextbeschreibung der Expertin oder des Experten, "
            "die Informationen aus den Feldern jobTitle, branches, skills, "
            "projectDescription und description enthält."
        )
    )

    # Tracks which structured input fields were used to build the full text.
    # Restricted to predefined field names for schema safety.
    full_text_fields_used: List[
        Literal["skills", "projectsDescription", "branches", "jobTitle", "description"]
    ] = Field(
        description=(
            "Eine Liste, die angibt, aus welchen Feldern die Informationen "
            "für den Volltext entnommen wurden."
        )
    )

In [ ]:
# Construct chat prompt template with predefined system and human messages.
# SYSTEM_PROMPT defines global behavior/instructions.
# HUMAN_PROMPT contains the user-specific input template.
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", HUMAN_PROMPT)
])

In [ ]:
# Wrap LLM with structured output schema (Pydantic model).
# Ensures responses conform to the Fulltext schema.
model_with_structure = model.with_structured_output(Fulltext)

### Process expert data

In [ ]:
import json
from tqdm import tqdm

In [ ]:
# Incremental file building tools

def load_existing_list(path: str):
    """Lädt eine bestehende JSON-Array-Datei. Fällt auf [] zurück, wenn Datei fehlt/leer/inkorrekt ist."""
    # Return empty list if file does not exist.
    if not os.path.exists(path):
        return []

    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

            # Ensure returned object is a list.
            if isinstance(data, list):
                return data

            # Fallback for unexpected JSON structure.
            return []

    except json.JSONDecodeError:
        # Treat empty or corrupted JSON as empty list.
        return []


def build_existing_id_set(items):
    """Extrahiert vorhandene IDs aus einer Liste von Experten-Objekten."""
    # Build a set of valid string IDs for fast lookup.
    ids = set()
    for it in items:
        _id = it.get("id")
        if isinstance(_id, str):
            ids.add(_id)
    return ids


def persist_list(path: str, items: list):
    """Schreibt die komplette Liste schön formatiert und UTF-8-sicher."""
    # Persist list atomically (overwrite) with readable formatting.
    with open(path, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

In [ ]:
# Paths
CLEANED_DATA = PATH + "expert_profiles_ready_for_enrichment.json" # Updated path to use the newly filtered data
ENHANCED_DATA = PATH + "enhanced_expert_profiles.json"

In [ ]:
# Load cleaned input data.
with open(CLEANED_DATA, "r", encoding="utf-8") as f:
    expert_data = json.load(f)
    print("Eingelesene Experten:", len(expert_data))

# Load existing enhanced output (if any) and build ID lookup set.
enhanced_experts = load_existing_list(ENHANCED_DATA)
existing_ids = build_existing_id_set(enhanced_experts)

print("Bereits vorhanden in OUTPUT:", len(enhanced_experts))


# Main processing loop (resume-safe).
for expert in tqdm(expert_data, desc="Enhance experts"):
    expert_id = expert.get("id")

    # Skip already processed experts.
    if expert_id in existing_ids:
        continue

    # Generate structured full-text representation via LLM.
    formatted_messages = chat_prompt.format_messages(expert=expert)
    response = model_with_structure.invoke(formatted_messages)

    # Merge generated fields into expert record.
    updated_expert = {
        **expert,
        "full_text": response.full_text,
        "full_text_fields_used": response.full_text_fields_used,
    }

    # Append immediately and persist incrementally (crash-safe).
    enhanced_experts.append(updated_expert)
    existing_ids.add(expert_id)
    persist_list(ENHANCED_DATA, enhanced_experts)

Eingelesene Experten: 307
Bereits vorhanden in OUTPUT: 307


Enhance experts: 100%|██████████| 307/307 [00:00<00:00, 479393.64it/s]


In [ ]:
# Load enhanced expert profiles from JSON file.
enhanced_expert_df = pd.read_json(PATH + "enhanced_expert_profiles.json")

# Display first rows for quick inspection.
print(enhanced_expert_df.head())

                                            branches companyLocationCity  \
0  [Technologie, Medien & Werbung, Unternehmensdi...          Lennestadt   
1                     [Unternehmensdienstleistungen]         Lüdenscheid   
2  [Automobilindustrie, Maschinenbau und Industri...         Lüdenscheid   
3  [Unternehmensdienstleistungen, Investor (Eigen...         Lüdenscheid   
4                      [Logistik, Konsum und Handel]         Ludenscheid   

   companyLocationStreet  companyLocationZip            companyName  \
0      Hagener Straße 64             57368.0        lenne.Tech GmbH   
1   Sauerfelder Straße 5             58511.0            RegioKonneX   
2   Sauerfelder Straße 5             58511.0            RegioKonneX   
3  Sauerfelder Straße 4              58511.0            RegioKonneX   
4   Wefelshohler Str 48a             58511.0  Karl Koerschulte GmbH   

       companyTypes      companyWebsite  \
0  [Dienstleistung]  https://lenne.tech   
1  [Dienstleistung]  www.regio

## Validate and fix fulltext und fields used data

In [ ]:
import numpy as np
import pandas as pd

def validate_and_fix_fulltext_fields_used(row):
    """
    Validiert und korrigiert die Liste 'full_text_fields_used' für einen Experten.

    - Entfernt Duplikate innerhalb der Liste.
    - Entfernt Feldnamen, die nicht im DataFrame existieren.
    - Entfernt Feldnamen, deren Werte leer, None oder NaN sind im Expertenobjekt.
    - Gibt eine aktualisierte Liste zurück und ggf. eine Fehlerbeschreibung.
    """

    # Extract original field list from row.
    original_list = row.get('full_text_fields_used')

    issues = []
    cleaned_fields = []

    # Validate expected list structure.
    if not isinstance(original_list, list):
        issues.append("'full_text_fields_used' ist keine Liste.")
        return {"fixed_list": [], "issues": issues}

    # Remove duplicates and normalize order for deterministic output.
    unique_fields = list(sorted(set(original_list)))

    for field_name in unique_fields:

        # Ensure referenced field exists in DataFrame schema.
        if field_name not in row.index:
            issues.append(
                f"Feld '{field_name}' in 'full_text_fields_used' existiert nicht im DataFrame."
            )
            continue

        field_value = row[field_name]
        is_field_invalid = False

        # Validate field value depending on type.
        if field_value is None:
            is_field_invalid = True

        elif isinstance(field_value, str):
            # Empty or whitespace-only string.
            if not field_value.strip():
                is_field_invalid = True

        elif isinstance(field_value, (list, np.ndarray)):
            # Empty list or array.
            if len(field_value) == 0:
                is_field_invalid = True

        elif pd.isna(field_value):
            # Generic NaN handling for scalar types.
            is_field_invalid = True

        # Keep valid fields, log invalid ones.
        if is_field_invalid:
            issues.append(
                f"Feld '{field_name}' ist leer oder None/NaN im Expertenobjekt – entfernt."
            )
        else:
            cleaned_fields.append(field_name)

    return {
        "fixed_list": cleaned_fields,
        "issues": issues if issues else "OK"
    }

In [ ]:
# Apply validation function row-wise.
results = enhanced_expert_df.apply(
    validate_and_fix_fulltext_fields_used,
    axis=1
)

# Update column with corrected field lists.
enhanced_expert_df['full_text_fields_used'] = results.apply(
    lambda x: x["fixed_list"]
)

# Store validation issues for auditing/debugging.
enhanced_expert_df['validation_issues'] = results.apply(
    lambda x: x["issues"]
)

# Inspect rows with validation issues.
issues_df = enhanced_expert_df[
    enhanced_expert_df['validation_issues'] != "OK"
]

if not issues_df.empty:
    print(f"Es wurden {len(issues_df)} Experten mit Validierungsproblemen gefunden:")

    # Print sample of problematic entries.
    for index, row in issues_df.head(10).iterrows():
        print(f"ID: {row['id']}")
        print(f"  full_text_fields_used (fixed): {row['full_text_fields_used']}")
        print(f"  Probleme: {row['validation_issues']}")
        print("\n")
else:
    print("Alle Experten haben die Validierung für 'full_text_fields_used' bestanden.")

Alle Experten haben die Validierung für 'full_text_fields_used' bestanden.


In [ ]:
import json

# Persist updated DataFrame to JSON (record-oriented, UTF-8 safe, pretty-printed).
enhanced_expert_df.to_json(
    ENHANCED_DATA,
    orient='records',
    indent=4,
    force_ascii=False
)

# Confirm successful write operation.
print(f"Der aktualisierte DataFrame wurde erfolgreich in '{ENHANCED_DATA}' gespeichert.")

Der aktualisierte DataFrame wurde erfolgreich in 'drive/MyDrive/Uni/Master/Masterthesis/Data/enhanced_expert_profiles.json' gespeichert.


## Short analysis of enhanced Data

- Hier werden die Länge der Volltextbeschreibung in Bezug auf die verwendeten Felder untersucht

In [ ]:
len(enhanced_expert_df)

307

In [ ]:
# Compute mean word count of generated full texts.
average_full_text_word_count = (
    enhanced_expert_df['full_text_word_count']
    .mean()
)

# Output formatted average.
print(
    "Die durchschnittliche Anzahl der Wörter im Feld 'full_text' beträgt: "
    f"{average_full_text_word_count:.2f}"
)

Die durchschnittliche Anzahl der Wörter im Feld 'full_text' beträgt: 72.08


In [ ]:
# Analyze full_text length depending on used input field combinations.

# Compute word count per full_text entry.
enhanced_expert_df['full_text_word_count'] = (
    enhanced_expert_df['full_text']
    .apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
)

# Normalize field combinations:
# - sort list for deterministic grouping
# - convert to tuple for hashability
enhanced_expert_df['full_text_fields_used_sorted'] = (
    enhanced_expert_df['full_text_fields_used']
    .apply(lambda x: tuple(sorted(x)) if isinstance(x, list) else ())
)

# Aggregate by field combination:
# - count number of experts
# - compute average word count
analysis_result = (
    enhanced_expert_df
    .groupby('full_text_fields_used_sorted')
    .agg(
        count=('id', 'count'),
        average_word_count=('full_text_word_count', 'mean')
    )
    .reset_index()
)

# Create readable string representation of field combinations.
analysis_result['used_fields_combination'] = (
    analysis_result['full_text_fields_used_sorted']
    .apply(lambda x: ', '.join(x) if isinstance(x, tuple) else '')
)

# Select relevant columns and sort by frequency.
analysis_result = (
    analysis_result[
        ['used_fields_combination', 'count', 'average_word_count']
    ]
    .sort_values(by='count', ascending=False)
)

# Display aggregated results.
display(analysis_result)

,used_fields_combination,count,average_word_count
2,"branches, description, jobTitle, projectsDescr...",99,92.767677
6,"branches, jobTitle, projectsDescription, skills",75,71.906667
7,"branches, jobTitle, skills",73,52.575342
3,"branches, description, jobTitle, skills",16,73.312500
8,"branches, projectsDescription, skills",11,56.272727
5,"branches, jobTitle, projectsDescription",7,52.285714
0,"branches, description, jobTitle",5,65.400000
4,"branches, description, projectsDescription, sk...",3,74.666667
1,"branches, description, jobTitle, projectsDescr...",3,61.000000
9,"branches, skills",3,60.000000


In [ ]:
for items in analysis_result["used_fields_combination"]:
  print(items)

branches, description, jobTitle, projectsDescription, skills
branches, jobTitle, projectsDescription, skills
branches, jobTitle, skills
branches, description, jobTitle, skills
branches, projectsDescription, skills
branches, jobTitle, projectsDescription
branches, description, jobTitle
branches, description, projectsDescription, skills
branches, description, jobTitle, projectsDescription
branches, skills
description, jobTitle
description, jobTitle, projectsDescription
description, jobTitle, projectsDescription, skills
jobTitle, projectsDescription
jobTitle, projectsDescription, skills
jobTitle, skills
projectsDescription, skills
